In [1]:
import cv2
import numpy as np
import imageio
import matplotlib.pyplot as plt



Features to add:
- Change appearance mode (light/dark)
- Video frame slider (across the bottom of the frame)
- Options panel (right side)
- Video filepath (read)
- exporting (+ filepath)
- frame skip rate


In [1]:
import tkinter
import tkinter.messagebox
import customtkinter
import cv2
from PIL import Image, ImageTk
import time
from screeninfo import get_monitors
import numpy as np



def CannyLines(frame, lower_bound=0, upper_bound=60, dilation=1):
    # canny edge detection on the grayscaled image
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) 
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    canny = cv2.Canny(blur, lower_bound, upper_bound)
    
    if dilation > 1: # if the line thickness we want is > 1 pixel
        kernel = np.ones((dilation, dilation), np.uint8) #also needs to be tuned
        img_dil = cv2.dilate(canny, kernel, iterations=1)
        return img_dil

    return canny

def MixedCannyLines(frame, ranges=[0,100,200,255], dilations = [1,2,4]):
    # grayscale and blur the images
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
  
    # template to which we keep adding the line images
    mixed_result = np.zeros_like(gray) 
    
    for idx, thickness in enumerate(dilations): # loop through all the line thicknesses we want

        canny = cv2.Canny(blur, ranges[idx], ranges[idx+1])  # 
        kernel = np.ones((thickness, thickness), np.uint8)  # make dilation kernel
        img_dil = cv2.dilate(canny, kernel, iterations=1)
        mixed_result = cv2.bitwise_or(mixed_result, img_dil)  # add the lines to the template image

    # reduce back to to 0-255 unit8 image
    mixed_result = np.clip(mixed_result, 0, 255)
    mixed_result = mixed_result.astype('uint8')
    
    return mixed_result



def blurImages(frame, prev_frame, blur=0.2):
    # convert frames to float values, to prevent numeric overflow with 8bit ints
    frame = np.asarray(frame, dtype=np.float32)
    prev_frame = np.asarray(prev_frame, dtype=np.float32) * blur
    
    blurred = frame + prev_frame

    # clip and convert back to 8bit int (so cv2 doesn't scream at me later)
    blurred = np.clip(blurred, 0, 255)
    blurred = blurred.astype('uint8')
    
    return blurred


customtkinter.set_appearance_mode("System")  # Modes: "System" (standard), "Dark", "Light"
customtkinter.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"


class GUI(customtkinter.CTk):
    def __init__(self):
        self.filePath = 'Input Videos\Fulham_goal.mp4'
        self.cap = cv2.VideoCapture(self.filePath)

        # get first frame
        ret, frame = self.cap.read()
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, (720,480))
        self.currentImage = frame
        self.prevImage =  np.zeros_like(frame, dtype=np.uint8)


        super().__init__()

        # Images settings
        self.imgParams = {'length' : int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT)),
                        'current frame' : 0,
                        'playing' : False, 
                        'blurring' : False, 
                            'blur coeff' : 0.2,
                        'multiLine' : False,
                            'lower bound' : 0, 
                            'upper bound' : 60, 
                            'dilation' : 1,
                            'ranges' : [0,100,200,255], 
                            'dilations' : [1,2,4],
                        'fadeout' : False}

        # configure window

        self.title("Graphic design is my passion.py")
        monitors = get_monitors()
        for m in monitors:
            if m.is_primary == True:
                width = m.width, 
                height = m.height

        self.geometry(f"{1100}x{580}")

        # configure grid layout (4x4)
        self.grid_columnconfigure(1, weight=1)
        self.grid_columnconfigure((2, 3), weight=0)
        self.grid_rowconfigure((0, 1, 2), weight=1)

        # SIDEBAR (column 0, all rows)
        self.sidebar_frame = customtkinter.CTkFrame(self, width=200, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(4, weight=1)

        text_var = tkinter.StringVar(value="Play Clip")
        self.play_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.play_switch_func())
        self.play_switch.grid(row=0, column=0, pady=10, padx=20, sticky="n")

        text_var = tkinter.StringVar(value="MultiLine")
        self.multiLine_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.left_panel_switches())
        self.multiLine_switch.grid(row=1, column=0, pady=10, padx=20, sticky="n")

        text_var = tkinter.StringVar(value="Blurring")
        self.blurring_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.left_panel_switches())
        self.blurring_switch.grid(row=2, column=0, pady=10, padx=20, sticky="n")

        text_var = tkinter.StringVar(value="Fadeout")
        self.fadeout_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.left_panel_switches())
        self.fadeout_switch.grid(row=3, column=0, pady=10, padx=20, sticky="n")


        # IMAGE FRAME
        self.image_frame = customtkinter.CTkFrame(self)
        self.image_frame.grid(row=0, column=1, rowspan=4, columnspan=2, padx=(10, 10), pady=(40, 40), sticky="nsew")
    
        imgtk = ImageTk.PhotoImage(image=Image.fromarray(frame))    
        self.panel = tkinter.Label(master=self.image_frame, image=imgtk, width=720, height=480)
        self.panel.image = imgtk
        self.panel.pack()


        # RIGHT-SIDE PANEL (tabs)
        self.tabview = customtkinter.CTkTabview(self, width=250)
        self.tabview.grid(row=0, column=3, rowspan=4, padx=(10, 10), pady=(0, 10), sticky="nsew")
        self.tabview.add("Edges")
        self.tabview.add("Blurring")
        self.tabview.add("Fadeout")
        # EDGES TAB
        self.label1 = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Lower Bound:").pack()
        self.lower_bound_txtbx = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), textvariable=tkinter.StringVar(value=0))
        self.lower_bound_txtbx.pack(pady=0, padx=10)
        self.label1 = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Upper Bound:").pack()
        self.upper_bound_txtbx = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), textvariable=tkinter.StringVar(value=100))
        self.upper_bound_txtbx.pack(pady=0, padx=10)
        self.label1 = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Dilation:").pack()
        self.line_dil_txtbx = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), textvariable=tkinter.StringVar(value=1))
        self.line_dil_txtbx.pack(pady=0, padx=10)
        self.main_button_1 = customtkinter.CTkButton(master=self.tabview.tab("Edges"), text="Update values", fg_color="transparent", border_width=2, text_color=("gray10", "#DCE4EE"), command=lambda: self.update_canny())
        self.main_button_1.pack(pady=30)


    def update_canny(self):
        print(self.imgParams)
        self.imgParams['lower bound'] = int(self.lower_bound_txtbx.get())
        self.imgParams['upper bound'] = int(self.upper_bound_txtbx.get())
        self.imgParams['dilation'] = int(self.line_dil_txtbx.get())
        print(self.imgParams)


    # formulate a better while loop than this ? 
    def left_panel_switches(self):
        self.imgParams['multiLine'] = self.multiLine_switch.get()
        self.imgParams['blurring'] = self.blurring_switch.get()
        self.imgParams['fadeout'] = self.fadeout_switch.get()
        #print(self.imgParams['fadeout'])

    
    def play_switch_func(self):
        # get their values
        update = self.play_switch.get()
        if update: self.refresh_frame()
        

    def refresh_frame(self):
        # load the next image here
        #print("refreshing")
        ret, frame = self.cap.read()
        if ret:
            #frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            frame = cv2.resize(frame, (720,480))
            self.augmentImage(frame)  
            self.update() 
            #time.sleep(0.1)       
            if self.play_switch.get(): self.refresh_frame() # call itself again, if we want that
        else:
            self.cap.release() # finished clip (restart it)
            self.cap = cv2.VideoCapture(self.filePath)
            if self.play_switch.get(): self.refresh_frame()


    def setFrame(self, frame): 
        imgtk = ImageTk.PhotoImage(image=Image.fromarray(frame))    
        self.panel.configure(image=imgtk)
        self.panel.image = imgtk
        #print("set new")

    def augmentImage(self, frame):
        if self.imgParams['multiLine'] == True:
            canny_lines = MixedCannyLines(frame)
        else:
            canny_lines = CannyLines(frame, lower_bound=self.imgParams['lower bound'],upper_bound=self.imgParams['upper bound'], dilation=self.imgParams['dilation'])

        if self.imgParams['blurring'] == True:
            result = blurImages(canny_lines, self.prevImage, blur=self.imgParams['blur coeff'])
        else:
            result = canny_lines

        self.setFrame(result)
        self.prevImage = result
        
        


    
if __name__ == "__main__":
    app = GUI()
    app.mainloop()

{'length': 158, 'current frame': 0, 'playing': False, 'blurring': False, 'blur coeff': 0.2, 'multiLine': False, 'lower bound': 0, 'upper bound': 60, 'dilation': 1, 'ranges': [0, 100, 200, 255], 'dilations': [1, 2, 4], 'fadeout': False}
{'length': 158, 'current frame': 0, 'playing': False, 'blurring': False, 'blur coeff': 0.2, 'multiLine': False, 'lower bound': 0, 'upper bound': 100, 'dilation': 1, 'ranges': [0, 100, 200, 255], 'dilations': [1, 2, 4], 'fadeout': False}
{'length': 158, 'current frame': 0, 'playing': False, 'blurring': False, 'blur coeff': 0.2, 'multiLine': False, 'lower bound': 0, 'upper bound': 100, 'dilation': 1, 'ranges': [0, 100, 200, 255], 'dilations': [1, 2, 4], 'fadeout': False}
{'length': 158, 'current frame': 0, 'playing': False, 'blurring': False, 'blur coeff': 0.2, 'multiLine': False, 'lower bound': 0, 'upper bound': 100, 'dilation': 2, 'ranges': [0, 100, 200, 255], 'dilations': [1, 2, 4], 'fadeout': False}
{'length': 158, 'current frame': 0, 'playing': False,

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\site-packages\customtkinter\windows\widgets\ctk_switch.py", line 405, in toggle
    self._command()
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_29800\2503173897.py", line 117, in <lambda>
    self.play_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.play_switch_func())
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_29800\2503173897.py", line 182, in play_switch_func
    if update: self.refresh_frame()
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_29800\2503173897.py", line 195, in refresh_frame
    if self.play_switch.get(): self.refresh_frame() # call itself again, if we want that
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_29800\2503173897.py", line 

In [16]:
from tkinter import *
import threading
import time

class gui:
    def __init__(self, window):
        # play button
        self.play_frame = Frame(master=window, relief=FLAT, borderwidth=1)
        self.play_frame.grid(row=0, column=0, padx=1, pady=1)
        self.play_button = Button(self.play_frame, text="play", fg="blue", command=lambda: self.play(1))
        self.play_button.pack()
        # stop button
        self.stop_frame = Frame(master=window, relief=FLAT, borderwidth=1)
        self.stop_frame.grid(row=0, column=2, padx=1, pady=1)
        self.stop_button = Button(self.stop_frame, text="stop", fg="red", command=lambda: self.play(0))
        self.stop_button.pack()

    def play(self, switch):
        a.set(switch) # update 'a'
        print(a.get())

root = Tk()

a = IntVar(value=0)

def tester(trig):
    while True:
        value = trig.get()
        if value == 1:
            time.sleep(0.5)
            print ("running")
        elif value == 0:
            time.sleep(0.5)
            print ("not running")

t1 = threading.Thread (target = tester, args = [a], daemon = True)
t1.start()

app = gui(root)
root.mainloop()

not running
not running
not running
not running
not running
not running
not running
not running
not running
not running
0
not running
not running
not running
not running
not running
1
not running
running
running
running
running
running
running
running
running
running
running
running
running


running


Exception in thread Thread-3 (tester):
Traceback (most recent call last):
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_8148\84278952.py", line 28, in tester
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\tkinter\__init__.py", line 568, in get
    value = self._tk.globalgetvar(self._name)
RuntimeError: main thread is not in main loop
